In [86]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [87]:
p_gwl = Path("../data/AquiMod_simobs_Gretna.csv")
p_met = Path("../data/ukcp18_simobs_Gretna.csv")
df_gwl = pd.read_csv(p_gwl, parse_dates=["Date"], dayfirst=True)
df_met = pd.read_csv(p_met, parse_dates=["Date"], dayfirst=True)
df_data = pd.merge(left=df_met, right=df_gwl, on=["Borehole", "Model", "Date"], how="inner").dropna()
df_data

,Borehole,Model,Date,precipwsnow,PET,Sim,Obs
11419,Gretna,AquiMod,1993-04-07,0.096710,1.530000,39.9447,40.084
11420,Gretna,AquiMod,1993-04-08,22.228661,1.530000,39.9812,40.082
11421,Gretna,AquiMod,1993-04-09,9.274128,1.530000,40.0087,40.106
11422,Gretna,AquiMod,1993-04-10,0.089421,1.530000,40.0106,40.121
11423,Gretna,AquiMod,1993-04-11,1.071286,1.530000,40.0064,40.135
...,...,...,...,...,...,...,...
20814,Gretna,AquiMod,2018-12-27,0.153541,0.248387,39.9721,39.959
20815,Gretna,AquiMod,2018-12-28,1.296672,0.248387,39.9695,39.953
20816,Gretna,AquiMod,2018-12-29,1.978836,0.248387,39.9697,39.950
20817,Gretna,AquiMod,2018-12-30,0.029849,0.248387,39.9671,39.941


In [88]:
fig1 = px.line(df_data, x="Date", y="Obs")
fig1.show()
fig2 = px.line(df_data, x="Date", y="precipwsnow")
fig2.show()
fig3 = px.line(df_data, x="Date", y="PET")
fig3.show()

In [89]:
precip = df_data["precipwsnow"].values
pet = df_data["PET"].values
gwl = df_data["Obs"].values

In [90]:
# Concatenate the features
features_arr = np.column_stack((precip, pet))
features_arr.shape

(9400, 2)

In [91]:
# Normalize the features
scaler = MinMaxScaler(feature_range=(-1, 1))
features_scaled_arr = scaler.fit_transform(features_arr)
features_scaled_arr.shape

(9400, 2)

In [92]:
# Normalise the target
target_scaler = MinMaxScaler(feature_range=(-1, 1))
gwl_scaled_arr = target_scaler.fit_transform(gwl.reshape(-1, 1))

In [93]:
def create_sequences(data, seq_length):
    """
    Transforms time-series data into sequences of a specified length.

    Parameters:
    data (np.array): A 2D numpy array where each row is a time step and each column is a feature.
    seq_length (int): The number of time steps to include in each output sequence.

    Returns:
    np.array: A 3D numpy array of shape (num_samples - seq_length + 1, seq_length, num_features).
    """

    xs = []  # Initialise an empty list to store sequences

    # For each possible sequence in the data...
    for i in range(len(data) - seq_length + 1):
        # Extract a sequence of length `seq_length`
        x = data[i:(i+seq_length)]
        # Append the sequence to the list
        xs.append(x)

    # Convert the list of sequences into a 3D numpy array
    return np.array(xs)

seq_length = 365
features_seq_arr = create_sequences(features_scaled_arr, seq_length)
# Also need to make sure the first 365 elements of the gwl array are clipped
gwl_arr = gwl_scaled_arr[seq_length - 1:]
print(features_seq_arr.shape)
print(gwl_arr.shape)

(9036, 365, 2)
(9036, 1)


In [94]:
features_tensor = torch.from_numpy(features_seq_arr).float()
gwl_tensor = torch.from_numpy(gwl_arr).float()#.unsqueeze(1)
print(features_tensor.shape)
print(gwl_tensor.shape)

torch.Size([9036, 365, 2])
torch.Size([9036, 1])


In [95]:
# Split into training and test sets
train_size = int(len(features_tensor) * 0.8)
test_size = len(features_tensor) - train_size

features_train = features_tensor[:train_size]
features_test = features_tensor[train_size:]
gwl_train  = gwl_tensor[:train_size]
gwl_test = gwl_tensor[train_size:]

print(f"{features_train.shape}: {gwl_train.shape}")
print(f"{features_test.shape}: {gwl_test.shape}")

torch.Size([7228, 365, 2]): torch.Size([7228, 1])
torch.Size([1808, 365, 2]): torch.Size([1808, 1])


In [96]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

# Work on this tomorrow
train_dataset = TimeSeriesDataset(features_train, gwl_train)
test_dataset = TimeSeriesDataset(features_test, gwl_test)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [97]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        # self.lstm.reset_parameters()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [98]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [99]:
# Initialize the model, loss function, and optimizer
model = LSTM(input_size=2, hidden_size=1, num_layers=1, output_size=1).to(device)

class NSELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, predictions: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        denominator = torch.sum((targets - torch.mean(targets)) ** 2)
        numerator = torch.sum((targets - predictions) ** 2)
        nse_val = numerator / denominator
        return nse_val

criterion = nn.MSELoss()
# criterion = NSELoss()

In [100]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [101]:
# Training function
def train_epoch(epoch: int):
    model.train()
    running_loss = 0
    for batch in train_loader:
        # batch is a list with two elements
        # The first element is the feature array
        # The second element is the GWL array
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        output = model(x_batch)
        loss = criterion(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss_across_batches = running_loss / len(test_loader)

    # print(f'Epoch: {epoch + 1} Final Batch, Loss: {running_loss}')
    return avg_loss_across_batches


In [102]:
# Testing
def test_epoch(epoch: int):
    model.eval()
    running_loss = 0
    for i, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        with torch.no_grad():
            output = model(x_batch)
            loss = criterion(output, y_batch)
            running_loss += loss.item()
    avg_loss_across_batches = running_loss / len(test_loader)
    
    # print(f'Epoch: {epoch + 1} Test Loss: {avg_loss_across_batches}')
    # print('***************************************************')
    return avg_loss_across_batches

In [103]:
def nse(predictions: np.ndarray, targets: np.ndarray) -> float:
    denominator = np.sum((targets - np.mean(targets)) ** 2)
    numerator = np.sum((targets - predictions) ** 2)
    nse_val = 1 - numerator / denominator
    return nse_val

### Impact of learning rate

#### hidden_size=50, num_layers=3, seq_length=200

LR|Epochs|Output Shape|Minimised|#Epochs at Minimum|Train NSE|Test NSE
--|------|-----|---------|---|---|---
0.001|100|flat line|True|30|NaN|NaN
0.0001|100|battlements|False|NaN|NaN|NaN
0.0001|500|hydrograph|False|NaN|0.73|0.34
0.0001|1000|hydrograph|True|1000|0.71|0.22
0.00001|1000|hydrograph|True|900|0.73|0.84
0.0005|100|battlements|False|NaN|0.05|-0.03


#### batch_size=16, hidden_size=1, num_layers=1, seq_length=200
LR|Epochs|Output Shape|Minimised|#Epochs at Minimum|Train NSE|Test NSE
--|------|-----|---------|---|---|---
0.001|500|hydrograph|False|NaN|0.5|0.3
0.002|500|hydrograph|False|NaN|0.55|0.35
0.005|500|battlements|True|60|0.07|-0.02
0.001|1000|hydrograph|False|NaN|0.60|0.35
0.001|2000|hydrograph|True|1350|0.65|0.43

#### hidden_size=2, num_layers=1, seq_length=200
LR|Epochs|Output Shape|Minimised|#Epochs at Minimum|Train NSE|Test NSE
--|------|-----|---------|---|---|---
0.001|1000|hydrograph|False|NaN|0.65|0.37
0.001|2000|hydrograph|False|NaN|0.71|0.70
0.001|4000|hydrograph|True|3860|0.79|0.86
0.001|5000|hydrograph|True|5000|0.75|0.24

Apparently increasing the num_layers can improve model training stability.

#### hidden_size=3, num_layers=2, seq_length=200
LR|Epochs|Output Shape|Minimised|#Epochs at Minimum|Train NSE|Test NSE
--|------|------------|---------|------------------|---------|--------
0.001|1000|hydrograph|False|NaN|0.76|0.11
0.001|4000|hydrograph|False|NaN|0.79|0.58
0.001|5000|hydrograph|True|5000|0.83|0.63

#### batch_size=64, hidden_size=3, num_layers=2, seq_length=365
LR|Epochs|Output Shape|Minimised|#Epochs at Minimum|Train NSE|Test NSE
--|------|------------|---------|------------------|---------|--------
0.001|2000|hydrograph|False|NaN|0.81|0.66
0.001|3000|hydrograph|True|2000|0.81|0.66

#### batch_size=64, hidden_size=1, num_layers=1, seq_length=365
LR|Epochs|Output Shape|Minimised|#Epochs at Minimum|Train NSE|Test NSE
--|------|------------|---------|------------------|---------|--------
0.001|100|sine|False|NaN|0.11|0.04
0.001|200|mountains|False|NaN|0.27|0.28
0.001|300|hydrograph|True|260|0.53|0.44
0.001|400|hydrograph|True|260|0.53|0.44
0.001|500|battlements|False|NaN|0.30|0.40
0.001|600|mountains|False|NaN|0.50|0.45
0.001|1100|hydrograph|False|NaN|0.67|0.59
0.001|1500|hydrograph|True|1450|0.68|0.62
0.0001|continued from previous|hydrograph|False|NaN|0.72|0.68
0.0001|1700|hydrograph|False|NaN|0.73|0.69
0.0001|1900|hydrograph|False|NaN|0.74|0.71
0.0001|2100|hydrograph|False|NaN|0.74|0.72
0.0001|2300|hydrograph|False|NaN|0.74|0.74
0.0001|2600|hydrograph|False|NaN|0.75|0.76
0.0001|2900|hydrograph|False|NaN|0.75|0.77
0.0001|3400|hydrograph|False|NaN|0.76|0.80
0.0001|3900|hydrograph|False|NaN|0.77|0.84
0.0001|4900|hydrograph|False|NaN|0.79|0.87
0.0001|5900|hydrograph|False|NaN|0.80|0.89
0.0001|6900|hydrograph|False|NaN|0.81|0.91
0.0001|7900|hydrograph|False|NaN|0.82|0.90
0.0001|7900|hydrograph|False|NaN|0.82|0.90
0.0001|9900|hydrograph|False|NaN|0.84|0.90
0.0001|11900|hydrograph|True|11600|0.84|0.90

Essentially, what we can gather here is that training is a difficult process. Training and performance is sensitive to random initialisation of parameeters. Some training runs find good solutions quickly whereas others get stuck in local optima.

Could write a function to start with a user-defined number of initial training attempts. These will run for a user-defined number of epochs. Then the best of them is selected for further training. (10 attempts of 1,000 epochs)

For some reason, it is also better to train in 100 or 1000 epoch stages, because when running for a large number of epochs the training just stagnates. This is something to do with the Adam optimiser, probably.

In [140]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# scheduler = StepLR(optimizer, step_size=50, gamma=0.5)
# scheduler = ReduceLROnPlateau(optimizer, verbose=True)
num_epochs = 2000
train_loss = []
test_loss = []

gwl_train_inverse = target_scaler.inverse_transform(gwl_train).flatten()
gwl_test_inverse = target_scaler.inverse_transform(gwl_test).flatten()

for epoch in range(num_epochs):
    train_loss.append(train_epoch(epoch))
    test_loss.append(test_epoch(epoch))
    # scheduler.step(train_loss[-1])
    with torch.no_grad():
        train_predicted = model(features_train.to(device)).to('cpu').numpy()
        test_predicted = model(features_test.to(device)).to('cpu').numpy()
    
    # Scale normalised data back to real-world GWL

    predicted_train_inverse = target_scaler.inverse_transform(train_predicted)
    predicted_test_inverse = target_scaler.inverse_transform(test_predicted)

    # Calculate NSE based on unscaled data
    train_nse = nse(predicted_train_inverse.flatten(), gwl_train_inverse)
    test_nse = nse(predicted_test_inverse.flatten(), gwl_test_inverse)
    print(f"Epoch {epoch + 1} Train NSE: {train_nse}")
    print(f"Epoch {epoch + 1}  Test NSE: {test_nse}", end=None)
    

    if (epoch + 1) != num_epochs:
        continue
    df = pd.DataFrame()
    df["Training Observed"] = np.append(gwl_train_inverse.flatten(), np.full(len(gwl_test), np.nan))
    df["Training Predicted"] = np.append(predicted_train_inverse.flatten(), np.full(len(gwl_test), np.nan))
    df["Testing Predicted"] = np.append(np.full(len(gwl_train), np.nan), predicted_test_inverse)
    df["Testing Observed"] = np.append(np.full(len(gwl_train), np.nan), gwl_test_inverse)
    fig = px.line(df)
    fig.show()

    fig = px.line(np.array(train_loss))
    fig.show()
    fig = px.line(np.array(test_loss))
    fig.show()


Epoch 1 Train NSE: 0.8363641904917795
Epoch 1  Test NSE: 0.8997237596341275
Epoch 2 Train NSE: 0.8364899354166133
Epoch 2  Test NSE: 0.9001030843814721
Epoch 3 Train NSE: 0.8365084264112632
Epoch 3  Test NSE: 0.9001198314102512
Epoch 4 Train NSE: 0.8364359049995383
Epoch 4  Test NSE: 0.9000217894680522
Epoch 5 Train NSE: 0.8363794637284776
Epoch 5  Test NSE: 0.8999336967213508
Epoch 6 Train NSE: 0.8363272208595538
Epoch 6  Test NSE: 0.8998591398855654
Epoch 7 Train NSE: 0.8362963542875635
Epoch 7  Test NSE: 0.8998082014397988
Epoch 8 Train NSE: 0.8362748045154795
Epoch 8  Test NSE: 0.8997726451777428
Epoch 9 Train NSE: 0.836258728815927
Epoch 9  Test NSE: 0.899747991703249
Epoch 10 Train NSE: 0.8362472271460686
Epoch 10  Test NSE: 0.8997270934916864
Epoch 11 Train NSE: 0.8362363964945395
Epoch 11  Test NSE: 0.8997115536631155
Epoch 12 Train NSE: 0.836235019908643
Epoch 12  Test NSE: 0.8997029481195034
Epoch 13 Train NSE: 0.8362304596030646
Epoch 13  Test NSE: 0.8996956694392133
Epoch 1

In [142]:
torch.save(model.state_dict(), 'temp.pt')
torch.save(model.state_dict(), 'seq365_hid1_lay1.pt')
# model.load_state_dict(torch.load('temp.pt'))

In [38]:
def count_parameters(model: LSTM) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model)} parameters")

The model has 184 parameters


In [126]:
# # Create animation of LSTM learning
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# # scheduler = ReduceLROnPlateau(optimizer, patience=50, verbose=True)
# num_epochs = 1000
# train_loss = []
# gwl_train_inverse = target_scaler.inverse_transform(gwl_train).flatten()
# incremental_results = []
# for epoch in range(num_epochs):
#     train_loss.append(train_epoch(epoch))
#     # scheduler.step(train_loss[-1])
#     with torch.no_grad():
#         train_predicted = model(features_train.to(device)).to('cpu').numpy()
    
#     predicted_train_inverse = target_scaler.inverse_transform(train_predicted)

#     train_nse = nse(predicted_train_inverse.flatten(), gwl_train_inverse)
#     print(f"Epoch {epoch + 1} Train NSE: {train_nse}")

#     incremental_results.append(predicted_train_inverse.flatten())

Epoch 1 Train NSE: -1.3385634208098645
Epoch 2 Train NSE: -0.11320718262148999
Epoch 3 Train NSE: 0.007474194471750684
Epoch 4 Train NSE: 0.013426120016032828
Epoch 5 Train NSE: 0.02457866165795697
Epoch 6 Train NSE: 0.04051046995534724
Epoch 7 Train NSE: 0.05686643935159286
Epoch 8 Train NSE: 0.0724952138903644
Epoch 9 Train NSE: 0.08412110085500646
Epoch 10 Train NSE: 0.09157379140645738
Epoch 11 Train NSE: 0.0972722177479648
Epoch 12 Train NSE: 0.1052909132219213
Epoch 13 Train NSE: 0.1359957041155898
Epoch 14 Train NSE: 0.1947546757001245
Epoch 15 Train NSE: 0.07365840817967129
Epoch 16 Train NSE: 0.0773326071065038
Epoch 17 Train NSE: 0.08295989125674919
Epoch 18 Train NSE: 0.08677541023648061
Epoch 19 Train NSE: 0.08950078414310225
Epoch 20 Train NSE: 0.09152848755501397
Epoch 21 Train NSE: 0.0931014222304688
Epoch 22 Train NSE: 0.09437575722060154
Epoch 23 Train NSE: 0.09545147262514087
Epoch 24 Train NSE: 0.09639366276590089
Epoch 25 Train NSE: 0.09724599213159157
Epoch 26 Trai

In [2]:
# fig = px.line(np.array(train_loss))
# fig.show()

In [1]:
# df_predicted = pd.DataFrame(incremental_results).T
# df_melted = df_predicted.reset_index().melt(id_vars="index", var_name="epoch", value_name="gwl")
# fig = px.line(df_melted, x="index", y="gwl", animation_frame="epoch")
# fig.add_trace(go.Scatter(y=gwl_train_inverse))
# fig.show()

In [129]:
# fig.write_html("how_machines_learn.html")